In [1]:
import sys
import numpy as np
from tqdm import tqdm_notebook
import pickle
import pandas as pd
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import time
import init_common as x
import simulate_corona_multi as sc
import simulate_corona_senior_multi as scs
import multiprocessing as mp
import save_part as sp
import helpers as hp

## setting
# 0 = one fixed value, 1
alpha_const = True
alpha_val = 0.7
R0 = x.R0_default

Trec_const = False
beta_mixed = True

## setup
out_filename_dir,out_fig_dir,out_stat_dir=hp.setup_paths(R0,alpha_val)
out_filename = out_filename_dir+"/"+x.out_filename
# choose distribution for beta and gammma
if beta_mixed:
    if Trec_const:
        import init_spec_beta as xx
    else:
        import init_spec_beta_gamma as xx
else:
    import init_spec_gamma as xx
# simulation function
if inout:
    import simulate_corona_inout as sc
else:
    import simulate_corona as sc
# alpha-strategy
if alpha_const:
    alpha_mat = np.full(shape=[x.N_locs,x.N_per],alpha_val)
else:
    idx_sel = np.arange(0,x.N_locs)
    init_sel = .7
    tar_sel = 1
    per = 50
    init_rem = 1
    tar_rem = 1
    alpha_mat = hp.create_alpha_matrix(idx_sel,init_sel,tar_sel,per,init_rem,tar_rem)
# get random vectors and rescale R0 if needed
R0_scale = R0/x.R0_default
beta_list,Trec_list = get_vectors(R0_scale)

## initialize
np.random.seed(123)
rep_vec = np.arange(0,x.N_simul)

def run_simulation(beta_list,Trec_list,alpha_mat):
    fun = partial(xx.simul,beta_list,Trec_list,alpha_mat)
    cores_cnt = mp.cpu_count()
    pool = mp.Pool(cores_cnt)
    tic = time.perf_counter()
    res = pool.map(fun,rep_vec)
    toc = time.perf_counter()
    pool.close()
    dt_avg = (toc-tic)/x.N_simul
    return res,dt_avg 

def save_results(filename,data,stat=False):
    if stat:       
        pct_S_mean = hp.mean_list(data,'sus')
        pct_I_mean = hp.mean_list(data,'inf')
        pct_R_mean = hp.mean_list(data,'rec')
        pct_S_std = hp.std_list(data,'sus')
        pct_I_std = hp.std_list(data,'inf')
        pct_R_std = hp.std_list(data,'rec')
        data = pd.DataFrame(list(zip(pct_S_mean, pct_I_mean, pct_R_mean, pct_S_std,pct_I_std,pct_R_std)), 
                         columns = ['sus_mean','inf_mean','rec_mean','sus_std','inf_std','rec_std'])
    fid = open(filename+".pickle",'wb')
    pickle.dump(data,fid)
    fid.close()
    

Trec_const = False
beta_mixed = True

if alpha_const:
    print("Alpha: "+str(np.round(alpha_val,3)))
else
    print("Alpha: plan")
print("R0:"+str(np.round(R0,3))+"\t (default value"+str(np.round(x.R0_default)))
print("Recovery Time constant:"+str(Trec_const))
print("Beta tail follows Power distribution:"+str(beta_mixed))

print("Running simulation ... ",end="",flush=True)
data,tpi=run_simulation(alpha_vec,fnc_name)
    print("Finished")
    print("Time per iteration (sec.): "+str(tpi))

    ## save results
    print("Saving results ...",end="",flush=True)
    save_results(out_filename_pct0+file_ext,pct_data,stat=False)
    print("Finished")    
    

Option type: 1
R0:2.2
alpha:1
beta:0.22000000000000003
Running simulation ... 

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
## statistics
if alpha_option_type==0:
    n = round(mp.cpu_count()/alpha_len)
    for ia in range(alpha_len):
        y = pct_data[ia*n:(ia+1)*n]
        alp = int(alpha_val[i]*100)
        save_results(x.out_filename_pct0+"_alpha_"+str(alp),y,stat=True)
else:
    save_results(x.out_filename_pct0+file_ext+"_stat",pct_data,stat=True)